# Logistic Regression for Sentiment Analysis with Theano

This is a tutorial on logistic regression using Theano based on the MNST 
logistic regression code provided at http://deeplearning.net/tutorial/logreg.html.
We make some code to read Andrew Mass' distribution of IMBD data
and make some changes in the original MNST Theano tutorial as needed. 
You will need to familiarize yourself with Theano to understand this tutorial well.
Take a look here: http://deeplearning.net/software/theano/tutorial/

# Simple Example

It will probably help if you consider this simpler code for logistic regression,
This code is provided at: http://deeplearning.net/software/theano/tutorial/examples.html.
Maybe try changing values of "N" and the number of "training_steps" and see what you get.
Later, we will use logistic regression too, but with a technique called "stochastic gradient descent." 

In [1]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random
##############
N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.rand(N, feats).astype(theano.config.floatX), rng.randint(size=N, low=0, high=2))
training_steps = 100
#np.asarray(your_data, dtype=theano.config.floatX)

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")
#print b.eval()
print("Initial model:")
#print(w.get_value())
#print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)),
          allow_input_downcast=True) # added downcasting...
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
#print(w.get_value())
#print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))
#----------------------------------------------------------

Initial model:
Final model:
target values for D:
[1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1
 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 0
 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0
 0 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0
 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1
 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1
 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 1
 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1
 1 1 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0
 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0]
prediction on D:
[0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1
 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 1 

Using gpu device 0: GeForce GT 750M


In [ ]:
# Now try the code with different values of "training_steps" and see what you get.
# For example, you can try:
# training_steps= 100, training_steps=500, training_steps=10000

# Logistic Regression with Theano, using Stochastic Gradient Descent

In [3]:
"""
This tutorial introduces logistic regression using Theano and stochastic
gradient descent.

Logistic regression is a probabilistic, linear classifier. It is parametrized
by a weight matrix :math:`W` and a bias vector :math:`b`. Classification is
done by projecting data points onto a set of hyperplanes, the distance to
which is used to determine a class membership probability.

Mathematically, this can be written as:

.. math::
  P(Y=i|x, W,b) &= softmax_i(W x + b) \\
                &= \frac {e^{W_i x + b_i}} {\sum_j e^{W_j x + b_j}}


The output of the model or prediction is then done by taking the argmax of
the vector whose i'th element is P(Y=i|x).

.. math::

  y_{pred} = argmax_i P(Y=i|x,W,b)


This tutorial presents a stochastic gradient descent optimization method
suitable for large datasets.


References:

    - textbooks: "Pattern Recognition and Machine Learning" -
                 Christopher M. Bishop, section 4.3.2

"""
from collections import namedtuple, defaultdict
from random import shuffle, randint
#----------------------------------------------------
__docformat__ = 'restructedtext en'

import cPickle
import gzip
import os
import sys
import timeit

import numpy
import numpy as np
import theano
import theano.tensor as T
#----------------------------------------------------

def get_data():
    '''
    
    '''
    all_data = []  
    DataDoc= namedtuple('DataDoc', 'tag words')
    with open('/Users/mam/CORE/RESEARCH/DEEPLEARNING/Doc2Vec/data/aclImdb/alldata-id.txt') as alldata:
        for line_no, line in enumerate(alldata):
            label=line.split()[0]
            word_list=line.lower().split()[1:]
            all_data.append(DataDoc(label, word_list))
    train_data = all_data[:25000]
    dev_data = all_data[25000:27500]+all_data[47500:50000]
    test_data=all_data[27500:47500]
    # labels
    train_tags=[ 1.0 for i in range(12500)] + [ 0.0 for i in range(12500)]
    dev_tags= [ 1.0 for i in range(2500)] + [ 0.0 for i in range(2500)]
    test_tags= [ 1.0 for i in range(10000)] + [ 0.0 for i in range(10000)]
    return train_data, train_tags, dev_data, dev_tags, test_data, test_tags
    #--------------------------------------------------
#train_data, train_tags, dev_data, dev_tags, test_data, test_tags=get_data()
########################


# Let's get a dictionary of all the words in training data
# These will be our bag-of-words features
# We won't need this function, since we will use gensim's built-in method "Dictionary" from the corpus module
# --> corpora.Dictionary, but we provide this so that you are clear on one way of how to do this.
def get_space(train_data):
    """
    input is a list of namedtuples
    get a dict of word space
    key=word
    value=len of the dict at that point 
    (that will be the index of the word and it is unique since the dict grows as we loop)
    """
    word_space=defaultdict(int)
    for doc in train_data:
        for w in doc.words:
            # indexes of words won't be in sequential order as they occur in data (can you tell why?), 
            # but that doesn't matter.
            word_space[w]+=1
    return word_space

# train_data, train_tags, dev_data, dev_tags, test_data, test_tags=get_data()
# word_space=get_space(train_data)
# word_space={w: word_space[w] for w in word_space if word_space[w] > 500}
# space_len=len(word_space)
# print "space_len: ", space_len
def get_sparse_vec(data_point, space):
    # create empty vector
    sparse_vec = np.zeros((len(space)))
    for w in set(data_point.words):
        # use exception handling such that this function can also be used to vectorize 
        # data with words not in train (i.e., test and dev data)
        try:
            sparse_vec[space[w]]=1
        except:
            continue
    return sparse_vec

   
# train_vecs= [get_sparse_vec(data_point, word_space) for data_point in train_data]
# test_vecs= [get_sparse_vec(data_point, word_space) for data_point in test_data]
# dev_vecs= [get_sparse_vec(data_point, word_space) for data_point in dev_data]
# #---------------------------
# train_vecs=np.array(train_vecs)
# train_tags=np.array(train_tags)
# dev_vecs=np.array(dev_vecs)
# dev_tags=np.array(dev_tags)
# test_vecs=np.array(test_vecs)
# test_tags=np.array(test_tags)
# print train_vecs.shape
# print dev_vecs.shape
# print test_vecs.shape


def load_data(train_vecs, train_tags, dev_vecs, dev_tags, test_vecs, test_tags):
    #------------------------------
    # Modified from Theano tutorial.
    # I basically pass data_x, data_y instead of data_xy
    def shared_dataset(data_x, data_y, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        shared_x = theano.shared(numpy.asarray(data_x,
                                               dtype=theano.config.floatX), borrow=borrow)
        shared_y = theano.shared(numpy.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        # When storing data on the GPU it has to be stored as floats
        # therefore we will store the labels as ``floatX`` as well
        # (``shared_y`` does exactly that). But during our computations
        # we need them as ints (we use labels as index, and if they are
        # floats it doesn't make sense) therefore instead of returning
        # ``shared_y`` we will have to cast it to int. This little hack
        # lets ous get around this issue
        return shared_x, T.cast(shared_y, 'int32')
    #-----------------------------------------------------------------
    train_set_x, train_set_y = shared_dataset(train_vecs, train_tags)
    valid_set_x, valid_set_y = shared_dataset(dev_vecs, dev_tags)
    test_set_x, test_set_y = shared_dataset(test_vecs, test_tags)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

#rval=load_data(train_vecs, train_tags)
#print rval


class LogisticRegression(object):
    """Multi-class Logistic Regression Class

    The logistic regression is fully described by a weight matrix :math:`W`
    and bias vector :math:`b`. Classification is done by projecting data
    points onto a set of hyperplanes, the distance to which is used to
    determine a class membership probability.
    """

    def __init__(self, input, n_in, n_out):
        """ Initialize the parameters of the logistic regression

        :type input: theano.tensor.TensorType
        :param input: symbolic variable that describes the input of the
                      architecture (one minibatch)

        :type n_in: int
        :param n_in: number of input units, the dimension of the space in
                     which the datapoints lie

        :type n_out: int
        :param n_out: number of output units, the dimension of the space in
                      which the labels lie

        """
        # start-snippet-1
        # initialize with 0 the weights W as a matrix of shape (n_in, n_out)
        self.W = theano.shared(
            value=numpy.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='W',
            borrow=True
        )
        # initialize the biases b as a vector of n_out 0s
        self.b = theano.shared(
            value=numpy.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )

        # symbolic expression for computing the matrix of class-membership
        # probabilities
        # Where:
        # W is a matrix where column-k represent the separation hyperplane for
        # class-k
        # x is a matrix where row-j  represents input training sample-j
        # b is a vector where element-k represent the free parameter of
        # hyperplane-k
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        # end-snippet-1

        # parameters of the model
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

    def negative_log_likelihood(self, y):
        """Return the mean of the negative log-likelihood of the prediction
        of this model under a given target distribution.

        .. math::

            \frac{1}{|\mathcal{D}|} \mathcal{L} (\theta=\{W,b\}, \mathcal{D}) =
            \frac{1}{|\mathcal{D}|} \sum_{i=0}^{|\mathcal{D}|}
                \log(P(Y=y^{(i)}|x^{(i)}, W,b)) \\
            \ell (\theta=\{W,b\}, \mathcal{D})

        :type y: theano.tensor.TensorType
        :param y: corresponds to a vector that gives for each example the
                  correct label

        Note: we use the mean instead of the sum so that
              the learning rate is less dependent on the batch size
        """
        # start-snippet-2
        # y.shape[0] is (symbolically) the number of rows in y, i.e.,
        # number of examples (call it n) in the minibatch
        # T.arange(y.shape[0]) is a symbolic vector which will contain
        # [0,1,2,... n-1] T.log(self.p_y_given_x) is a matrix of
        # Log-Probabilities (call it LP) with one row per example and
        # one column per class LP[T.arange(y.shape[0]),y] is a vector
        # v containing [LP[0,y[0]], LP[1,y[1]], LP[2,y[2]], ...,
        # LP[n-1,y[n-1]]] and T.mean(LP[T.arange(y.shape[0]),y]) is
        # the mean (across minibatch examples) of the elements in v,
        # i.e., the mean log-likelihood across the minibatch.
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
        # end-snippet-2

    def errors(self, y):
        """Return a float representing the number of errors in the minibatch
        over the total number of examples of the minibatch ; zero one
        loss over the size of the minibatch

        :type y: theano.tensor.TensorType
        :param y: corresponds to a vector that gives for each example the
                  correct label
        """

        # check if y has same dimension of y_pred
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        # check if y is of the correct datatype
        if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()


def sgd_optimization(learning_rate=0.13, n_epochs=1000,
                           batch_size=600):
    """
    Demonstrate stochastic gradient descent optimization of a log-linear
    model

    This is demonstrated on MNIST.

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
                          gradient)

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz

    """
    datasets=load_data(train_vecs, train_tags, dev_vecs, dev_tags, test_vecs, test_tags)
    train_set_x, train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    print train_set_x.shape[0]
    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch

    # generate symbolic variables for input (x and y represent a
    # minibatch)
    x = T.matrix('x')  # data, presented as rasterized images
    y = T.ivector('y')  # labels, presented as 1D vector of [int] labels

    # construct the logistic regression class
    # Each MNIST image has size 28*28
    # MAM: We change size: n_in=space_len
    classifier = LogisticRegression(input=x, n_in=space_len, n_out=2)

    # the cost we minimize during training is the negative log likelihood of
    # the model in symbolic format
    cost = classifier.negative_log_likelihood(y)

    # compiling a Theano function that computes the mistakes that are made by
    # the model on a minibatch
    test_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    # compute the gradient of cost with respect to theta = (W,b)
    g_W = T.grad(cost=cost, wrt=classifier.W)
    g_b = T.grad(cost=cost, wrt=classifier.b)

    # start-snippet-3
    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs.
    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]

    # compiling a Theano function `train_model` that returns the cost, but in
    # the same time updates the parameter of the model based on the rules
    # defined in `updates`
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    # end-snippet-3

    ###############
    # TRAIN MODEL #
    ###############
    print '... training the model'
    # early-stopping parameters
    patience = 5000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                                  # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                  # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    test_score = 0.
    start_time = timeit.default_timer()

    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i)
                                     for i in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    # test it on the test set

                    test_losses = [test_model(i)
                                   for i in xrange(n_test_batches)]
                    test_score = numpy.mean(test_losses)

                    print(
                        (
                            '     epoch %i, minibatch %i/%i, test error of'
                            ' best model %f %%'
                        ) %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )

                    # save the best model
                    with open('best_model.pkl', 'w') as f:
                        cPickle.dump(classifier, f)

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    print(
        (
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )
        % (best_validation_loss * 100., test_score * 100.)
    )
    print 'The code run for %d epochs, with %f epochs/sec' % (
        epoch, 1. * epoch / (end_time - start_time))
    print ('The code for file ' +
                          'best_model.pkl' +
                          ' ran for %.1fs' % ((end_time - start_time)))


def predict():
    """
    An example of how to load a trained model and use it
    to predict labels.
    """

    # load the saved model
    classifier = cPickle.load(open('best_model.pkl'))

    # compile a predictor function
    predict_model = theano.function(
        inputs=[classifier.input],
        outputs=classifier.y_pred)

    # We can test it on some examples from test test
    datasets=load_data(train_vecs, train_tags, dev_vecs, dev_tags, test_vecs, test_tags)
    #train_set_x, train_set_y = datasets[0]
    #valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    test_set_x = test_set_x.get_value()
    predicted_values = predict_model(test_set_x[:10])
    print ("Predicted values for the first 10 examples in test set:")
    print predicted_values


if __name__ == '__main__':
    train_data, train_tags, dev_data, dev_tags, test_data, test_tags=get_data()
    word_space=get_space(train_data)
    # We only use words with a give frequency threshold. This also helps much with memory.
    # Note that I am using a *very* high threshold here, since the goal is to demonstrate 
    # the technique and its utility, rather than higher accuracy. I also had some memory issues.
    word_space={w: word_space[w] for w in word_space if word_space[w] > 150} 
    space_len=len(word_space)
    print("space_len: %d" % space_len)
    train_vecs= [get_sparse_vec(data_point, word_space) for data_point in train_data]
    test_vecs= [get_sparse_vec(data_point, word_space) for data_point in test_data]
    dev_vecs= [get_sparse_vec(data_point, word_space) for data_point in dev_data]
    #del word_space
    #---------------------------
    train_vecs=np.array(train_vecs)
    train_tags=np.array(train_tags)
    dev_vecs=np.array(dev_vecs)
    dev_tags=np.array(dev_tags)
    test_vecs=np.array(test_vecs)
    test_tags=np.array(test_tags)
    #del train_data, train_tags, dev_data, dev_tags, test_data, test_tags
    print('train_vecs.shape: %d, %d' % train_vecs.shape)
    print('dev_vecs.shape: %d, %d' % dev_vecs.shape)
    print('test_vecs.shape: %d, %d' % test_vecs.shape)
    sgd_optimization()
    #------------------------------------------------------
    print('Now predicting...')
    predict()

space_len: 3047
train_vecs.shape: 25000, 3047
dev_vecs.shape: 5000, 3047
test_vecs.shape: 20000, 3047
Subtensor{int64}.0
... building the model
... training the model
epoch 1, minibatch 41/41, validation error 52.083333 %
     epoch 1, minibatch 41/41, test error of best model 50.505051 %
epoch 2, minibatch 41/41, validation error 52.083333 %
epoch 3, minibatch 41/41, validation error 52.062500 %
     epoch 3, minibatch 41/41, test error of best model 50.505051 %
epoch 4, minibatch 41/41, validation error 51.895833 %
     epoch 4, minibatch 41/41, test error of best model 50.297980 %
epoch 5, minibatch 41/41, validation error 50.937500 %
     epoch 5, minibatch 41/41, test error of best model 49.560606 %
epoch 6, minibatch 41/41, validation error 49.416667 %
     epoch 6, minibatch 41/41, test error of best model 48.267677 %
epoch 7, minibatch 41/41, validation error 47.708333 %
     epoch 7, minibatch 41/41, test error of best model 46.616162 %
epoch 8, minibatch 41/41, validation err